In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from utils import Parameters
from retirement_problem import get_retirement_problem
from data import load_data
import cvxpy as cp

sns.set_theme(font_scale=1.5, style='whitegrid')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data = load_data()

deterministic = False

age_start = 60
age_end = 105
T = age_end - age_start
collar = False

d_max = 8

rho_B = 1.033
rho_I = 1.057
rho_R = 1.057

sex = 'F' 

B_init = 200.; shares = 1.; basis = B_init / 2; price = B_init
I_init = 400.
R_init = 200.

capital_tax_rate = 0.15

gamma = 500.

e = pd.Series(np.zeros(120), index=range(120))
a = pd.Series(np.zeros(120), index=range(120))
a[70:] = 3.938 * 12
l = pd.Series(np.zeros(120), index=range(120))
c0 = (B_init + I_init + R_init + a.loc[age_start:85].sum() + e[age_start:85].sum()) * 0.0375

parameters = Parameters(B_init=B_init,
                        I_init=I_init,
                        R_init=R_init,
                        basis=basis,
                        price=price,
                        shares=shares,
                        age_start=age_start,
                        age_end=age_end,
                        sex=sex,
                        collar=collar,
                        rho_B=rho_B,
                        rho_I=rho_I,
                        rho_R=rho_R,
                        e=e,
                        a=a,
                        l=l,
                        d_max=d_max,
                        c0=c0,
                        gamma=gamma,
                        w_stocks_B=None,
                        w_stocks_I=None,
                        w_stocks_R=None,
                        k=data.inflation.median(),
                        s1=None,
                        s2=None,
                        deterministic=deterministic,
                        capital_tax_rate=capital_tax_rate)


In [14]:
problem = get_retirement_problem(B_init=parameters.B_init,
                        I_init=parameters.I_init,
                        R_init=parameters.R_init,
                        basis=parameters.basis,
                        price=parameters.price,
                        rho_B=parameters.rho_B,
                        rho_I=parameters.rho_I,
                        rho_R=parameters.rho_R,
                        kappa=data.kappa.loc[age_start:age_start+T-1],
                        beta=data.beta,
                        eta=data.eta,
                        e=parameters.e.loc[age_start:age_start+T-1],
                        a=parameters.a.loc[age_start:age_start+T-1],
                        l=parameters.l.loc[age_start:age_start+T-1],
                        T=T,
                        d_max=parameters.d_max,
                        c0=parameters.c0,
                        gamma=parameters.gamma,
                        deterministic=parameters.deterministic,
                        capital_tax_rate=parameters.capital_tax_rate)

import time
kwargs = {'solver' : 'CLARABEL', 'verbose' : False}
a = time.time()
print(problem.problem.solve(**kwargs))
b = time.time()

problem.problem.solver_stats
print(f'Number of variables: , exact {sum([v.size for v in problem.problem.variables()])}, predicted {13 * T}')
constraints = problem.problem.constraints
equality_constraints = [con for con in constraints if isinstance(con, cp.constraints.Equality)]
inequality_constraints = [con for con in constraints if isinstance(con, cp.constraints.Inequality)]
print(f'Number of equality constrainta: exact {sum([con.size for con in equality_constraints])}, predicte {7 * T}')
print(f'Number of inequality constraints: exact {sum([con.size for con in inequality_constraints])}, predicted {9 * T}')
print(f'Solve time: {problem.problem.solver_stats.solve_time} s')


1463.7879813271138
Number of variables: , exact 590, predicted 585
Number of equality constrainta: exact 319, predicte 315
Number of inequality constraints: exact 408, predicted 405
Solve time: 0.009491833 s
